In [8]:
import pandas as pd
import os
from data_preparation import load_data
from tqdm import tqdm
import numpy as np
tqdm.pandas()
import tensorflow as tf

2023-04-09 15:57:02.043345: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 15:57:03.053638: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATA_FOLDER = "/mnt/d/sign-lang-data/"

In [3]:
train = pd.read_csv(os.path.join(DATA_FOLDER, "train_with_nb_frame.csv"))

In [4]:
lst_of_test_parquet = [
    "train_landmark_files/22343/1110225521.parquet",#330
    "train_landmark_files/4718/2748291567.parquet",#5
    "train_landmark_files/49445/2648133604.parquet",#60
    "train_landmark_files/62590/2817812959.parquet" #150
]

In [5]:
#train[train.nb_frame >= 150]

In [6]:
parquet_df = pd.read_parquet(os.path.join(DATA_FOLDER, "train_landmark_files/62590/2817812959.parquet"), engine="pyarrow")
parquet_df

,frame,row_id,type,landmark_index,x,y,z
0,60,60-face-0,face,0,0.490645,0.530036,-0.039722
1,60,60-face-1,face,1,0.484890,0.500506,-0.059791
2,60,60-face-2,face,2,0.486941,0.509968,-0.034611
3,60,60-face-3,face,3,0.474281,0.477195,-0.038282
4,60,60-face-4,face,4,0.484129,0.492718,-0.062013
...,...,...,...,...,...,...,...
49408,150,150-right_hand-16,right_hand,16,0.376999,0.859378,-0.095406
49409,150,150-right_hand-17,right_hand,17,0.269614,0.899242,-0.061837
49410,150,150-right_hand-18,right_hand,18,0.301550,0.898282,-0.080791
49411,150,150-right_hand-19,right_hand,19,0.328819,0.894403,-0.084535


In [42]:
frame_group_size = 15

for parquet in lst_of_test_parquet:
    parquet_df = pd.read_parquet(os.path.join(DATA_FOLDER, parquet), engine="pyarrow")
    parquet_df2 = parquet_df#[parquet_df.type == 'face']
    # do mean of 10 group in parquet_df2
    nbframe = len(parquet_df2["frame"].unique().tolist())
    fisrt_frame = parquet_df2["frame"].unique().min()
    last_frame = parquet_df2["frame"].unique().max()
    print(f"nbframe = {nbframe}")
    group_size = int(((nbframe - nbframe % frame_group_size) / frame_group_size))
    rest = nbframe % frame_group_size
    print(f"{nbframe} = {frame_group_size} x {group_size} + {rest}")

    if rest == 0:
        for i in range(frame_group_size):
            group = parquet_df2[(parquet_df2["frame"] >= i*group_size+fisrt_frame) & (parquet_df2["frame"] <= (i+1)*group_size+fisrt_frame)]
            # group by type and landmark_index and do mean
            group = group[["type", "landmark_index", "x", "y", "z"]]
            group = group.groupby(["type", "landmark_index"]).mean()
            #print(group.shape)

            p = pd.DataFrame(group[["x", "y", "z"]].values).fillna(0)
            d = tf.constant(p.values, dtype=tf.float32)
            d  = tf.concat(
                    [d, tf.zeros((7,3), dtype=tf.float32)], axis=0
                )
            d = tf.reshape(d, (22,25, 3)  )

    else:
        if group_size == 0:
            pass
        else:
            for i in range(frame_group_size):

                group = parquet_df2[(parquet_df2["frame"] >= i*group_size+fisrt_frame) & (parquet_df2["frame"] <= (i+1)*group_size+fisrt_frame)]
                group = group[["type", "landmark_index", "x", "y", "z"]]
                group = group.groupby(["type", "landmark_index"]).mean()
                p = pd.DataFrame(group[["x", "y", "z"]].values).fillna(0)
                d = tf.constant(p.values, dtype=tf.float32)
                d  = tf.concat(
                    [d, tf.zeros((7,3), dtype=tf.float32)], axis=0
                )
                d = tf.reshape(d, (22,25, 3))

            group = parquet_df2[(parquet_df2["frame"] >= 10 * group_size + fisrt_frame) & (parquet_df2["frame"] <= (10 + 1) * group_size + fisrt_frame + rest)]
            group = group[["type", "landmark_index", "x", "y", "z"]]
            group = group.groupby(["type", "landmark_index"]).mean()
            p = pd.DataFrame(group[["x", "y", "z"]].values).fillna(0)
            d = tf.constant(p.values, dtype=tf.float32)
            d  = tf.concat(
                    [d, tf.zeros((7,3), dtype=tf.float32)], axis=0
                )
            d = tf.reshape(d, (22,25, 3)  )


nbframe = 330
330 = 15 x 22 + 0
nbframe = 6
6 = 15 x 0 + 6
nbframe = 45
45 = 15 x 3 + 0
nbframe = 91
91 = 15 x 6 + 1
